In [18]:
# Anonymisera idrottarnas namn
import pandas as pd
import hashlib as hl

df = pd.read_csv("../Data/athlete_events.csv")
#df["Name"] = df["Name"].astype(str)
#df["Name"] = df["Name"].apply(lambda athlete_name: hl.sha256(athlete_name.encode()).hexdigest())
#df.head()

In [ ]:
# Rensa bort så att det bara blir en medalj per gren
df.groupby(["Event","Year"])["Medal"].size()
df.groupby(["Year"])["Medal"].size()


In [ ]:
# See vilka år som har vinter OS
#df.groupby("Year")["Medal"].size()

# Could be used to calculate chance of medal for each year
df.groupby("Year")[["NOC","Medal"]].count()
df.groupby("Year")[["NOC","Medal"]].count()
#df[ df["NOC"] == "HUN"  ].count()

,NOC,Medal
Year,,
1896,380,143
1900,1936,604
1904,1301,486
1906,1733,458
1908,3101,831
1912,4040,941
1920,4292,1308
1924,5693,962
1928,5574,823
